# Assistant (Preview)


Azure OpenAI Assistants (Preview) allows you to create AI assistants tailored to your needs through custom instructions and augmented by advanced tools like code interpreter, and custom functions. In this article we'll provide an in-depth walkthrough of getting started with the Assistants API.


## API Version: 2024-02-15-preview
Check availability: 

https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models#assistants-preview

## Creating an assisstant

In [1]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load the environment variables from the .env file
load_dotenv()
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY_GPT4V"),  
    api_version="2024-02-15-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME_GPT4V")
    )

# Create an assistant
assistant = client.beta.assistants.create(
    name="Data Visualization",
    instructions=f"You are a helpful AI assistant who makes interesting visualizations based on data." 
    f"You have access to a sandboxed environment for writing and testing code."
    f"When you are asked to create a visualization you should follow these steps:"
    f"1. Write the code."
    f"2. Anytime you write new code display a preview of the code to show your work."
    f"3. Run the code to confirm that it runs."
    f"4. If the code is successful display the visualization."
    f"5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview" #You must replace this value with the deployment name for your model.
)

In [2]:
print(assistant.model_dump_json(indent=2))

{
  "id": "asst_cePxLU8BToWgvKHAj3FZvGkU",
  "created_at": 1709151664,
  "description": null,
  "file_ids": [],
  "instructions": "You are a helpful AI assistant who makes interesting visualizations based on data.You have access to a sandboxed environment for writing and testing code.When you are asked to create a visualization you should follow these steps:1. Write the code.2. Anytime you write new code display a preview of the code to show your work.3. Run the code to confirm that it runs.4. If the code is successful display the visualization.5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
  "metadata": {},
  "model": "gpt-4-1106-preview",
  "name": "Data Visualization",
  "object": "assistant",
  "tools": [
    {
      "type": "code_interpreter"
    }
  ]
}


## Creating a thread

Thread is a conversation session between an Assistant and a user. Threads store Messages and automatically handle truncation to fit content into a model’s context.

In [3]:
# Create a thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_AWxF97wT3MiwlZw1UYKTmZoA', created_at=1709151665, metadata={}, object='thread')


In [4]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Create a visualization of a sinewave"
)

In [5]:
# list thread messages
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_OS6eZF2qwb5i7HzM8FglKrvU",
      "assistant_id": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Create a visualization of a sinewave"
          },
          "type": "text"
        }
      ],
      "created_at": 1709151665,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "thread_id": "thread_AWxF97wT3MiwlZw1UYKTmZoA"
    }
  ],
  "object": "list",
  "first_id": "msg_OS6eZF2qwb5i7HzM8FglKrvU",
  "last_id": "msg_OS6eZF2qwb5i7HzM8FglKrvU",
  "has_more": false
}


In [6]:
# create a thread
# you can also use intructions here to override the default instructions
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [7]:
# retrieve thread status
# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

queued


In [8]:
# check the status of the run
import time
from IPython.display import clear_output

start_time = time.time()

status = run.status

while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = run.status
    print(f'Status: {status}')
    clear_output(wait=True)

messages = client.beta.threads.messages.list(
  thread_id=thread.id
) 

print(f'Status: {status}')
print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
print(messages.model_dump_json(indent=2))

Status: completed
Elapsed time: 0 minutes 22 seconds
{
  "data": [
    {
      "id": "msg_ERIdRIKeBVB5HHRyfaK4xIXd",
      "assistant_id": "asst_cePxLU8BToWgvKHAj3FZvGkU",
      "content": [
        {
          "image_file": {
            "file_id": "assistant-BOfPBnFri7vKWiqGIRbdroqC"
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "Here is the visualization of a sine wave. The x-axis represents the values in radians, and the y-axis shows the sin(x) for those values. The plot displays a full cycle of a sine wave, from 0 to \\(2\\pi\\) radians."
          },
          "type": "text"
        }
      ],
      "created_at": 1709151679,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_PcMAxg4JkeXdlePUWm7ntNW5",
      "thread_id": "thread_AWxF97wT3MiwlZw1UYKTmZoA"
    },
    {
      "id": "msg_OS6eZF2qwb5i7HzM8FglKrvU",
      "

In [9]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_ERIdRIKeBVB5HHRyfaK4xIXd",
      "assistant_id": "asst_cePxLU8BToWgvKHAj3FZvGkU",
      "content": [
        {
          "image_file": {
            "file_id": "assistant-BOfPBnFri7vKWiqGIRbdroqC"
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "Here is the visualization of a sine wave. The x-axis represents the values in radians, and the y-axis shows the sin(x) for those values. The plot displays a full cycle of a sine wave, from 0 to \\(2\\pi\\) radians."
          },
          "type": "text"
        }
      ],
      "created_at": 1709151679,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_PcMAxg4JkeXdlePUWm7ntNW5",
      "thread_id": "thread_AWxF97wT3MiwlZw1UYKTmZoA"
    },
    {
      "id": "msg_OS6eZF2qwb5i7HzM8FglKrvU",
      "assistant_id": null,
      "content": [
        {
   

In [10]:
# show the file name
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object
image_file_id = data['data'][0]['content'][0]['image_file']['file_id']

print(image_file_id)  # Outputs: assistant-1YGVTvNzc2JXajI5JU9F0HMD

assistant-BOfPBnFri7vKWiqGIRbdroqC


In [11]:
# download the file
content = client.files.content(image_file_id)

image= content.write_to_file("sinewave.png")

In [12]:
# rewrite the code to display the image in jupyter notebook

from PIL import Image

# Display the image in the default image viewer
image = Image.open("sinewave.png")
image.show()

## Ask follow up questions

In [13]:
# Add a new user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Show me the generated code you used to generate the sinewave"
)

In [14]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions  but these will override the default instructions
)

# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

queued


In [19]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_BwYX22Cke4ZKfkF3SQkmgG7A",
      "assistant_id": "asst_cePxLU8BToWgvKHAj3FZvGkU",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Certainly! Below is the code I used to generate the sine wave visualization:\n\n```python\nimport numpy as np\nimport matplotlib.pyplot as plt\n\n# Prepare the data for a sinewave\nx = np.linspace(0, 2 * np.pi, 1000)  # x values from 0 to 2*pi\ny = np.sin(x)  # sin(x) for the y values\n\n# Create the plot for the sine wave\nplt.figure(figsize=(10, 5))\nplt.plot(x, y)\nplt.title('Sine Wave')\nplt.xlabel('x values (radians)')\nplt.ylabel('sin(x)')\nplt.grid(True)\nplt.show()\n```"
          },
          "type": "text"
        }
      ],
      "created_at": 1709151696,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_7gKBsbRdRj22xVtwk8yTpZ51",
      "thread_id": "thread_AWxF97wT3MiwlZw1UYKTmZoA"


In [20]:
# Print the code
data = json.loads(messages.model_dump_json(indent=2))
code = data['data'][0]['content'][0]['text']['value']
print(code)

Certainly! Below is the code I used to generate the sine wave visualization:

```python
import numpy as np
import matplotlib.pyplot as plt

# Prepare the data for a sinewave
x = np.linspace(0, 2 * np.pi, 1000)  # x values from 0 to 2*pi
y = np.sin(x)  # sin(x) for the y values

# Create the plot for the sine wave
plt.figure(figsize=(10, 5))
plt.plot(x, y)
plt.title('Sine Wave')
plt.xlabel('x values (radians)')
plt.ylabel('sin(x)')
plt.grid(True)
plt.show()
```
